# Objectif

Récuperer le fichier:
`https://github.com/VPerrollaz/immobilier/blob/master/donnees/brute.json`

puis transformer le en un dataset permettant de faire de l'apprentissage.

# Récupération fichier

In [2]:
from requests import get

In [6]:
ADRESSE = "https://raw.githubusercontent.com/VPerrollaz/immobilier/master/donnees/brute.json"

In [7]:
requete = get(ADRESSE)
requete.status_code

200

In [8]:
requete.reason

'OK'

In [9]:
contenu = requete.text

In [10]:
type(contenu)

str

In [11]:
contenu[:1000]

'{"id": "annonce-138905473-376235", "genre": "Appartement", "prix": "374 400 \\u20ac", "pcs": "3 p 2 ch 90 m\\u00b2", "desc": "Appartement type 3 - TOURS CATH\\u00c9DRALE TOURS CATH\\u00c9DRALE: Ravissant appartement type 3 d\'environ 90 m\\u00b2 dans r\\u00e9sidence de standing (~50 lots) avec cave et parking. Entr\\u00e9e,...\\n\\u00c7a m\'int\\u00e9resse", "lien": "https://www.seloger.com/annonces/achat/appartement/tours-37/cathedrale/138905473.htm"}\n{"id": "annonce-140620177-376235", "genre": "Appartement", "prix": "499 200 \\u20ac", "pcs": "5 p 4 ch 146,27 m\\u00b2", "desc": "TOURS HYPERCENTRE - Appartement TOURS HYPERCENTRE: Superbe appartement de 146,37 m\\u00b2 (LOI CARREZ) dans un immeuble de caract\\u00e8re avec ascenseur. Entr\\u00e9e, s\\u00e9jour SUD, vaste...\\n\\u00c7a m\'int\\u00e9resse", "lien": "https://www.seloger.com/annonces/achat/appartement/tours-37/centre/140620177.htm"}\n{"id": "annonce-140620179-376235", "genre": "Appartement", "prix": "499 200 \\u20ac", "pcs

In [12]:
contenu[-1000:]

'es avec jardin ou appartements du 2 au 3 pi\\u00e8ces: faites votre choix - Exceptionnel: une cuisine Blanchet d\'Huismes am\\u00e9nag\\u00e9e et install\\u00e9e dans...\\n\\u00c7a m\'int\\u00e9resse", "lien": "https://www.selogerneuf.com/annonces/achat/maison/tours-37/138636083/#?cmp=INTSL_ListToDetail"}\n{"id": "annonce-137873909-139524", "genre": "Boutique", "prix": "195 000 \\u20ac", "pcs": "173 m\\u00b2", "desc": "Place Saint \\u00c9loi, Local commercial de 173m\\u00b2 avec acc\\u00e8s PMR. Il comprend des espaces bureaux en rez-de-chauss\\u00e9e de 92m\\u00b2 environ et 79m\\u00b2 \\u00e0 l\'\\u00e9tage. Belle fa\\u00e7ade.\\n\\u00c7a m\'int\\u00e9resse", "lien": "https://www.seloger.com/annonces/achat/boutique/tours-37/137873909.htm"}\n{"id": "annonce-136457803-733454", "genre": "Parking", "prix": "15 000 \\u20ac", "pcs": "13 m\\u00b2", "desc": "GRAMMONT.PLACE DE LA LIBERT\\u00c9. Garage en sous-sol.\\n\\u00c7a m\'int\\u00e9resse", "lien": "https://www.seloger.com/annonces/acha

# Décodage du json

Délicat car construction à la main sans librairie de sérialisation (i.e. `pyserde`)!

In [13]:
import json

In [16]:
json.loads(contenu)

JSONDecodeError: Extra data: line 2 column 1 (char 445)

In [17]:
contenu[440:450]

'tm"}\n{"id"'

In [19]:
print(contenu[:450])

{"id": "annonce-138905473-376235", "genre": "Appartement", "prix": "374 400 \u20ac", "pcs": "3 p 2 ch 90 m\u00b2", "desc": "Appartement type 3 - TOURS CATH\u00c9DRALE TOURS CATH\u00c9DRALE: Ravissant appartement type 3 d'environ 90 m\u00b2 dans r\u00e9sidence de standing (~50 lots) avec cave et parking. Entr\u00e9e,...\n\u00c7a m'int\u00e9resse", "lien": "https://www.seloger.com/annonces/achat/appartement/tours-37/cathedrale/138905473.htm"}
{"id"


On voit qu'on a juste une juxtaposition d'annonces représentées par des dictionnaires.
Il faut rajouter les virgules et les crochets pour transformer le contenu en une liste de dictionnaires.

**REMARQUE** le `\\u00b2` représente 6 caractères alors que `\u00b2` représente un caractère via sa numérotation unicode!

In [20]:
print("\\u00b2")

\u00b2


In [21]:
print("\u00b2")

²


In [26]:
annonces = json.loads("[" + contenu[:-1].replace("\n", ",") + "]")

In [27]:
type(annonces)

list

In [29]:
type(annonces[0])

dict

In [30]:
annonces[0]

{'id': 'annonce-138905473-376235',
 'genre': 'Appartement',
 'prix': '374 400 €',
 'pcs': '3 p 2 ch 90 m²',
 'desc': "Appartement type 3 - TOURS CATHÉDRALE TOURS CATHÉDRALE: Ravissant appartement type 3 d'environ 90 m² dans résidence de standing (~50 lots) avec cave et parking. Entrée,...\nÇa m'intéresse",
 'lien': 'https://www.seloger.com/annonces/achat/appartement/tours-37/cathedrale/138905473.htm'}

In [31]:
len(annonces)

1818

# Extraire l'information pertinente des colonnes

e.g. 
- id (str), 
- genre (catégorie), 
- prix (int), 
- surface (int), 
- pieces (int), 
- neuf (bool), 
- quartier (categorie?)
- ?

In [32]:
import pandas as pd

In [34]:
df = pd.DataFrame(annonces)

In [35]:
df.describe()

,id,genre,prix,pcs,desc,lien
count,1818,1818,1818,1818,1818,1818
unique,1798,16,1152,1140,1591,1798
top,annonce-139534407-749291,Appartement,155 000 €,3 p 2 ch 65 m²,"Livraison: Octobre 2020 - Rare, plein centre d...",https://www.selogerneuf.com/annonces/investiss...
freq,2,1041,11,30,19,2


In [36]:
df.head()

,id,genre,prix,pcs,desc,lien
0,annonce-138905473-376235,Appartement,374 400 €,3 p 2 ch 90 m²,Appartement type 3 - TOURS CATHÉDRALE TOURS CA...,https://www.seloger.com/annonces/achat/apparte...
1,annonce-140620177-376235,Appartement,499 200 €,"5 p 4 ch 146,27 m²",TOURS HYPERCENTRE - Appartement TOURS HYPERCEN...,https://www.seloger.com/annonces/achat/apparte...
2,annonce-140620179-376235,Appartement,499 200 €,5 p 3 ch 110 m²,TOURS PRÉBENDES NORD - APPARTEMENT TOURS PRÉBE...,https://www.seloger.com/annonces/achat/apparte...
3,annonce-133494153-376235,Maison / Villa,508 000 €,6 p 4 ch 132 m²,TOURS PRÉBENDES - PARTICULIER TOURANGEAUX TOUR...,https://www.seloger.com/annonces/achat/maison/...
4,annonce-137425993-376235,Maison / Villa,676 000 €,7 p 5 ch 185 m²,TOURS STRASBOURG / RABELAIS - Maison TOURS STR...,https://www.seloger.com/annonces/achat-de-pres...


In [37]:
df.dtypes

id       object
genre    object
prix     object
pcs      object
desc     object
lien     object
dtype: object

In [38]:
df.genre.unique()

array(['Appartement', 'Maison / Villa', 'Appartement neuf',
       'Projet de construction', 'Maison / Villa neuve', 'Bâtiment',
       'Parking', 'Bureau', 'Local commercial', 'Immeuble', 'Boutique',
       'Terrain', 'Divers', 'Loft/Atelier/Surface', 'Château',
       'Hôtel particulier'], dtype=object)

In [39]:
equivalent_appartements = {'Appartement', 'Appartement neuf'}
equivalent_maisons = {'Maison / Villa', 'Maison / Villa neuve', 'Château', 'Hôtel particulier'}

In [42]:
df["appartement"] = df.eval("genre in @equivalent_appartements")

In [43]:
df["maison"] = df.eval("genre in @equivalent_maisons")

In [49]:
df["a_supprimer"] = df.eval("not (maison | appartement)")

In [50]:
df.describe()

,id,genre,prix,pcs,desc,lien,appartement,maison,a_supprimer
count,1818,1818,1818,1818,1818,1818,1818,1818,1818
unique,1798,16,1152,1140,1591,1798,2,2,2
top,annonce-139534407-749291,Appartement,155 000 €,3 p 2 ch 65 m²,"Livraison: Octobre 2020 - Rare, plein centre d...",https://www.selogerneuf.com/annonces/investiss...,True,False,False
freq,2,1041,11,30,19,2,1253,1406,1665


In [52]:
equivalent_neufs = {'Appartement neuf', 'Maison / Villa neuve'}
df["neuf"] = df.eval("genre in @equivalent_neufs")

In [72]:
masque = df["prix"].str.replace("€", "").str.replace(" ", "").str.replace("HT", "").str.replace("HH", "").str.isdecimal()

In [73]:
df["prix"].loc[masque == False]

843    €
Name: prix, dtype: object